In [1]:
from datetime import timedelta, date
import fetch
from keplergl import KeplerGl
import pandas as pd
import sqlite3

In [2]:
def getMonth(curr_month):
    month = fetch.get_weather(usecols='time,lat,lon,avg(temp_K)', datetime=date(2015,curr_month, 1), groupBy='lat,lon')
    conn = sqlite3.connect('../database/database.db')
    if curr_month == 12:
        monthlen = (date(2016, 1, 1) - date(2015, curr_month, 1)).days - 1
    else:
        monthlen = (date(2015, curr_month + 1, 1) - date(2015, curr_month, 1)).days -1

    for currDay in (date(2015,curr_month, 2) + timedelta(n) for n in range(monthlen)):
        df = fetch.get_weather(usecols='time, lat, lon, avg(temp_K)', datetime=currDay, groupBy='lat, lon', conn=conn)
        if not df.empty:
            month = pd.concat([month, df], ignore_index=True)
            
    month['time'] = month['time'].apply(lambda x: pd.Timestamp(x[:10] + ' ' + x[11:]))

    conn.close()

    return month

In [3]:
months = [None]

for i in range(1, 13):
    months.append(getMonth(i))

In [7]:
%run config.py

In [20]:
monthlyMap = KeplerGl(height=600,data={'month': months[1]}, config=dailyAvgTemp)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [19]:
day = fetch.get_weather(usecols='time,lat,lon,temp_K', datetime=date(2015,6,22))
day['time'] = day['time'].apply(lambda x: pd.Timestamp(x[:10] + ' ' + x[11:]))
day['temp_K'] = day['temp_K'].apply(lambda x: (x-273)*1.8 +32)

# The Macro Impacts of Micro-Climates on the Energy Consumption of Urban Buildings

In [ ]:
dailyMap = KeplerGl(height=600,data={'day': day}, config=dailyTemp)
dailyMap